# Treatment Checker Demo

This is a brief walkthrough demonstrating the project's functionality. The project was not initially written as a notebook as I prefer to keep the path to production as short as possible in general.
Check out `treatment_checker.py` to see the full functionality.

In [28]:
import sys
from loguru import logger
from treatment_checker.treatment_checker import main

sys.path.append("../treatment_checker/")  # Path stuff for Jupyter.
logger.remove(0)  # Disable logging for the demonstration.

### **Example 1:** Bespoke Full-Context Advice
In this example, the system uses information from the drugs information database and the patient history database to advise on a proposed treatment for a specific patient. In this particular patient's medical history, they reporting not responding well to ibuprofen. The system successfully retrieved data from multiple sources is able to offer effective advice on the matter as a result.

To achieve this result, the original query is rephrased into several sub-queries to get a better match with existing embeddings. These subqueries are each used for retrieval, each returning several documents. The document ranks are conbined with reciprocal rank fusion, to find the most relevent.

In [31]:
main("Should patient 001 be prescribed ibuprofen for their tooth ache?")

Based on the drug information context and patient history context provided, patient 001 should not be prescribed ibuprofen for their toothache. The patient history indicates that when prescribed ibuprofen for pain management in the past for an ear infection, the patient reported that ibuprofen was not effective in managing the pain. Therefore, considering the patient's previous experience with ibuprofen and its ineffectiveness, an alternative pain management strategy should be considered for the patient's toothache.


### **Example 2:** Avoiding a Costly Mistake
The system again generates an answer involving drug information from the database and context from the spefic patient's medical history. The answer advises against using aspirin, avoiding a potential mistake. This would not have been possible without the context provided by the retrieval.

Identifying the correct patient file to retrieve based on natural language is key to this process and can be troublesome. To aid with this, we use prompt engineering. The relevent patient is identified correctly based on a prompt that contains specific instructions regarding patient identification. This could likely be further improved by *few-shot prompting* and employing more sophisticated embedding techniques.

In [38]:
main("Should we prescribe aspirin for patient 003's headache?")

Based on the drug information context and patient history context provided, it is not advisable to prescribe aspirin for patient 003's headache. Aspirin is not recommended for patients with hypertension, and patient 003 was diagnosed with hypertension in 2018. Additionally, the patient is currently pregnant, and it is important to avoid medications that may pose risks during pregnancy. Considering both pieces of context, it is best to explore alternative treatment options for patient 003's headache that are safe for a pregnant individual with a history of hypertension.


### **Example 3:** Interrogating Patient History Only
Here the system correctly identifies the query as being related only to the patient history. It invokes only the patient history RAG chain, thus significantly reducing latency when compared to invoking the full service chain.

Latency is a serious concern in RAG systems, so it makes sense to modularise the service where possible. Here only naive RAG is executed.

In [36]:
main("Does patient 002 have issues with any common medications?")

Yes, patient 002 has a severe allergic reaction to penicillin, so they should avoid medications containing penicillin.


### **Example 4:** General Drug Advice Only
In this final example, the system once again correctly routes the query to the appropriate RAG chain. This time retieving only information from the drugs database.

In [39]:
main("When should Warafin be prescribed?")

Warfarin should be prescribed to treat or prevent blood clots in veins or arteries, which can reduce the risk of stroke, heart attack, or other serious conditions. It should be prescribed by a doctor after considering the patient's medical history and condition.


### Summary
We can see how using techniques such as prompt engineering, RAG, reciprocal rank fusion and semantic query routing can yield good results despite using `chat-gpt-3.5`. We saw the system take a natural language query, route it to the proper sub-system, then provide nuanced answers utilising privately held data. Thereby using RAG to provide a low-cost and flexible alternative to fine-tuning an LLM.

We could imagine how such a system could be applied in practice, utilising a far larger database of medications produced by the company and more comprehensive patient histories.

It would of course be vitally important to test the system both from the engineering perspective using standard methods such as unit and integration testing and the AI perspective, perhaps using RAGAS.